In [ ]:
!pip install tensorflow

# Importación de librerías y comprobación de GPU

In [1]:
import tensorflow as tf

import numpy as np
import os
import time
import pandas as pd


2025-11-11 02:46:10.252173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762829170.445248      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762829170.501076      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
print("GPU Disponible:", tf.config.list_physical_devices('GPU'))

GPU Disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
!nvidia-smi

Tue Nov 11 02:46:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             26W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Carga de datos

Carga de datos de diálogos de South Park

In [4]:

# Cargar el archivo
df = pd.read_csv("/kaggle/input/southparklines/All-seasons.csv")

# Si quieres ver las columnas:
print(df.columns)

# Quedarte solo con la columna de diálogos
lines = df["Line"]

# Limpiar saltos de línea y espacios extra
lines = lines.str.replace(r'\n', ' ', regex=True).str.strip()

# Concatenar todo en un solo string
southParkDialog = str(" ".join(lines))

print(southParkDialog[:500])

Index(['Season', 'Episode', 'Character', 'Line'], dtype='object')
You guys, you guys! Chef is going away. Going away? For how long? Forever. I'm sorry boys. Chef said he's been bored, so he joining a group called the Super Adventure Club. Wow! Chef?? What kind of questions do you think adventuring around the world is gonna answer?! What's the meaning of life? Why are we here? I hope you're making the right choice. I'm gonna miss him.  I'm gonna miss Chef and I...and I don't know how to tell him! Dude, how are we gonna go on? Chef was our fuh...f-ffriend. And w


Carga de datos de entrenamiento para chat bots

In [5]:
# Cargar el archivo
df = pd.read_csv("/kaggle/input/3k-conversations-dataset-for-chatbot/Conversation.csv")

# Si quieres ver las columnas:
print(df.columns)

# Quedarte solo con la columna de diálogos
lines = df["question"]

# Limpiar saltos de línea y espacios extra
lines = lines.str.replace(r'\n', ' ', regex=True).str.strip()

# Concatenar todo en un solo string
conversation = str(" ".join(lines))

print(conversation[:500])

Index(['Unnamed: 0', 'question', 'answer'], dtype='object')
hi, how are you doing? i'm fine. how about yourself? i'm pretty good. thanks for asking. no problem. so how have you been? i've been great. what about you? i've been good. i'm in school right now. what school do you go to? i go to pcc. do you like it there? it's okay. it's a really big campus. good luck with school. how's it going? i'm doing well. how about you? never better, thanks. so how have you been lately? i've actually been pretty good. you? i'm actually in school right now. which school 


Cargar datos de entrenamiento para chatbots, datos de conversaciones humanas, y oraciones extraídas de wikipedia, y combinar todos los datasets.

In [6]:
# Read, then decode for py2 compat.
chatbot = open("/kaggle/input/chatbot-training-dataset/chatbot dataset.txt", 'rb').read(1000000).decode(encoding='utf-8')
human = open("/kaggle/input/human-conversation-training-data/human_chat.txt", 'rb').read(1000000).decode(encoding='utf-8')
wikipedia = open("/kaggle/input/wikipedia-sentences/wikisent2.txt", 'rb').read(150000).decode(encoding='utf-8')


text = chatbot+human+wikipedia[:150000]+southParkDialog[:150000]+conversation

# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 585415 characters


In [7]:
print(text[:250])
vocab = sorted(set(text))
print(f'\n{len(vocab)} unique characters')

What are your interests	I am interested in all kinds of things. We can talk about anything!
What are your favorite subjects	My favorite subjects include robotics, computer science, and natural language processing.
What are your interests	I am interes

114 unique characters


# Procesamiento de datos


In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), 
    invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)#.numpy()


In [10]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(585415,), dtype=int64, numpy=array([56, 71, 64, ..., 77, 67, 17])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

W
h
a
t
 
a
r
e
 
y


In [13]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'W' b'h' b'a' b't' b' ' b'a' b'r' b'e' b' ' b'y' b'o' b'u' b'r' b' '
 b'i' b'n' b't' b'e' b'r' b'e' b's' b't' b's' b'\t' b'I' b' ' b'a' b'm'
 b' ' b'i' b'n' b't' b'e' b'r' b'e' b's' b't' b'e' b'd' b' ' b'i' b'n'
 b' ' b'a' b'l' b'l' b' ' b'k' b'i' b'n' b'd' b's' b' ' b'o' b'f' b' '
 b't' b'h' b'i' b'n' b'g' b's' b'.' b' ' b'W' b'e' b' ' b'c' b'a' b'n'
 b' ' b't' b'a' b'l' b'k' b' ' b'a' b'b' b'o' b'u' b't' b' ' b'a' b'n'
 b'y' b't' b'h' b'i' b'n' b'g' b'!' b'\n' b'W' b'h' b'a' b't' b' ' b'a'
 b'r' b'e' b' '], shape=(101,), dtype=string)


In [14]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'What are your interests\tI am interested in all kinds of things. We can talk about anything!\nWhat are '
b'your favorite subjects\tMy favorite subjects include robotics, computer science, and natural language '
b'processing.\nWhat are your interests\tI am interested in a wide variety of topics, and read rather a lo'
b"t.\nWhat is your number\tI don't have any number\nWhat is your number\t23 skiddoo!\nWhat is your favorite "
b"number\tI find I'm quite fond of the number 42.\nWhat can you eat\tI consume RAM, and binary digits.\nWhy"


In [15]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [16]:
dataset = sequences.map(split_input_target)

In [17]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'What are your interests\tI am interested in all kinds of things. We can talk about anything!\nWhat are'
Target: b'hat are your interests\tI am interested in all kinds of things. We can talk about anything!\nWhat are '


# Training batches

In [18]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [19]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)

    if states is None:
        # The LSTM layer will initialize states to zero by default when initial_state is None
        lstm_output, hidden_state, cell_state = self.lstm(x, training=training)
        states = [hidden_state, cell_state] # LSTM returns a list of states
        x = lstm_output # Assign the output sequences to x
    else:
        # Use the provided states
        lstm_output, hidden_state, cell_state = self.lstm(x, initial_state=states, training=training)
        states = [hidden_state, cell_state] # LSTM returns a list of states
        x = lstm_output # Assign the output sequences to x


    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [20]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 2048

model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

I0000 00:00:1762829435.489521     105 cuda_dnn.cc:529] Loaded cuDNN version 90300


(64, 100, 115) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 100, 512)         │        58,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ((64, 100, 2048), (64, │    20,979,712 │
│                                 │ 2048), (64, 2048))     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 100, 115)         │       235,635 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,274,227 (81.15 MB)

 Trainable params: 21,274,227 (81.15 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [24]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"haracters are going to die, and there's nothing we can do to save them? Yes. And it won't be long be"

Next Char Predictions:
 b'M9\xe2\x80\x9c\tS;m;k\xf0\x9f\x98\x9bsTB\xe2\x80\x999Ld\xef\xbc\x89\xf0\x9f\x99\x82F1&\n5 I0\xef\xbc\x89za)\xef\xbc\x9aq\xef\xbc\x893,\xe6\xb9\x98s-TD*kMA1\'s"zE\tM1\nqX_2eELR\xc3\x82)[\xc3\x82q?a\xe2\x80\x94\\\xf0\x9f\x98\x9b\xf0\x9f\x98\x86+b\xe2\x80\x94\xe2\x80\x99\xc3\xb1Gi\xf0\x9f\x98\x82\xc3\xb1E[i\xc3\x829Bd"\tU::q \xf0\x9f\x98\x90]K'


In [25]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [26]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 115)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.7447033, shape=(), dtype=float32)


In [27]:
tf.exp(example_batch_mean_loss).numpy()

114.97369

In [28]:
model.compile(optimizer='adam', loss=loss)

In [29]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [30]:
EPOCHS = 40

In [31]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 15s 139ms/step - loss: 3.5816
Epoch 2/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 13s 139ms/step - loss: 2.2772
Epoch 3/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 13s 140ms/step - loss: 1.9671
Epoch 4/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 13s 140ms/step - loss: 1.7407
Epoch 5/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 13s 140ms/step - loss: 1.5667
Epoch 6/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 141ms/step - loss: 1.4448
Epoch 7/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 141ms/step - loss: 1.3482
Epoch 8/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 142ms/step - loss: 1.2647
Epoch 9/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - loss: 1.1875
Epoch 10/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - loss: 1.1003
Epoch 11/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - loss: 1.0214
Epoch 12/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - loss: 0.9354
Epoch 13/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - loss: 0.8500
Epoch 14/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 143ms/step - loss: 0.7544
Epoch 15/40
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 

In [32]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states


In [33]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [34]:
start = time.time()
states = None
next_char = tf.constant(['Hello, how are you?'])
result = [next_char]

for n in range(200):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80) #[0].numpy().decode('utf-8')
print('\nRun time:', end - start)

Hello, how are you?
Human 1: do you have a featuring joa
Human 2: yeah. I'm going to Lain in a mud of a geologive blow. Oh my Chusckey, like "Hello. He's bored. She clubbed their pockers? Yeah, everyone trying to warn u 

________________________________________________________________________________

Run time: 0.9639086723327637


In [52]:
chat = True
result = tf.constant([''])
states = None
# next_char = tf.constant(['Hello, how are you?'])
# result = [next_char]

print('Welcome to the ChatBot, it is trained on data from Wikipedia, South Park, and both human and robot conversations!\nTo exit just type "exit"')

while chat:
    response = ['']
    prompt = tf.constant([input('')])
    if prompt == 'exit':
        chat = False
        break
    result = [result[0] + prompt]
    # print('result:', result)
    generate = True
    while generate:
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result = [result[0] + next_char]
        response = [response[0] + next_char]
        if next_char == '.' or next_char == '?' or next_char == '!':
            generate = False
    # print('result2:', result)
    response = tf.strings.join(response)
    
    print(response[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

    

Welcome to the ChatBot, it is trained on data from Wikipedia, South Park, and both human and robot conversations!
To exit just type "exit"


 hello!



Human 1: Wow! 

________________________________________________________________________________


 How are you?


 Auch make you defeated the Evily Gads? 

________________________________________________________________________________


...

 And what do you think about that?


 Oh come bring a game to riding a bug immertant and Sauper in biture already are invented handless are performed with the Guentest Daves. 

________________________________________________________________________________


 Thank you for your time.



100 Park was an amisonamity 70 gie signic vagina system in pass area only on the InDplant Adely English, shaw aircomplis stage of its hybrids off beit's swort and pergent partical in and directed by Musum Phalita apactmed on Oh director Williams and London and non-starring Marian Channel 5, ticected 😛
10 mile Pur is a dark asteroid from the inner regions of the asteroid better thing to a Hindu release in 2006 by Giancia" Bost and Jieway Etink. 

________________________________________________________________________________


 exit


In [52]:
print(result[0], '\n\n' + '_'*80)

tf.Tensor(b"Hello, how are you?! Don't you are not ready? Ready. All rich, funday of it. Jesus Christ.  Ts yes we need to work about the weekend!\nHuman 1: Oh no\xe2\x80\xa6 I see. What would I do it in Scort?\nHuman 1: Oh too. If you could before you are weirding\nHuman 1: Hi!\nHuman 2: Hi there\nHuman 1: How is it going?\nHuman 2: Goo\nHuman 1: Hi!\nHuman 2: Hi there. I hope it's delicious little chicten about things, you've been so bad beat hands.\nHuman 1:  I hope that interesting if I was.\nDo you make meeting\tyou are been up working, and sleep in commonly which am going magazine.  It doesn't what I think they're believe in 9/11 once and meet you. I... are you went into?\nHuman 2: Anything tell you have to explain to that beautiful money? I'm different's for 9/11 and then operate the World of Radio Minest Basslovelit, Chef!! We can't look in ready here too. If we don't need some good reason, that's a good name into talking any carofocking. you like you?! I'm a Good Gack as gettin